# Part I. 
## ETL Pipeline for Pre-Processing the Files
### CODE FOR PRE-PROCESSING THE FILES
#### Import Python packages 

In [69]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [70]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join("/home/workspace/event_data/*.csv"))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [71]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [72]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. 
## Apache Cassandra coding portion. 
### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the event_datafile_new.csv after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [73]:
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])

    # The cluster needs a session connection to begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [74]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [75]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## The following will give three queries

### Query 1
The aim of query 1 is to make a query that can provide you with the artist, song title, and the song's length for a specific sessionId and itemInSession. This is done by creating a table with a composite PRIMARY KEY, consisting of 'session_id, item_in_session'. 

In [76]:
query = "CREATE TABLE IF NOT EXISTS song_length "
query = query + """(session_id int, 
                    item_in_session int, 
                    artist_name text, 
                    song_title text, 
                    song_length float, 
                    PRIMARY KEY (session_id, item_in_session));"""

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Opens the csv file and making an INSERT query

In [77]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_length (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT query to verify that the INSERT was correct

In [78]:
query = """SELECT artist_name, 
                  song_title, 
                  song_length 
             FROM song_length 
            WHERE session_id = 338 
              AND item_in_session = 4;"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
The second query gives the artist, song title, and first and last name of the user, based on a specific userId and sessionId. The song titles are ordered by the itemInSession. The PRIMARY KEY would then consist of the partition key, '(user_id, session_id)', and the clustering key, 'item_in_session'. 


In [79]:
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + """(user_id int, 
                    session_id int, 
                    item_in_session int, 
                    artist_name text, 
                    song_title text, 
                    first_name text, 
                    last_name text, 
                    PRIMARY KEY ((user_id, session_id), item_in_session));"""

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Opens the csv file and making an INSERT query

In [80]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_session (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[9]))    

#### SELECT query to verify that the INSERT was correct

In [81]:
query = """SELECT artist_name, 
                  song_title, 
                  first_name, 
                  last_name 
             FROM user_session 
            WHERE user_id = 10 
              AND session_id = 182;"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Keep On Keepin' On
Three Drives Greece 2000 Sylvie Greece 2000
Sebastien Tellier Kilometer Sylvie Kilometer
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### Query 3
This query provides you with the first and last name of every user who listens to a specific song. The PRIMARY KEY would then consist of a composite key, '(song_title, user_id)', where 'song_title' is the partition key, and 'user_id' is the clustering key. This way the query will select each individual user who listened to each song. It is important to use user_id instead of first and last name since a name is not unique. 

In [82]:
query = "CREATE TABLE IF NOT EXISTS user_from_song "
query = query + """(song_title text, 
                    user_id int, 
                    first_name text, 
                    last_name text, 
                    PRIMARY KEY (song_title, user_id));"""

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Opens the csv file and making an INSERT query

In [83]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_from_song (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT query to verify that the INSERT was correct

In [65]:
query = "SELECT first_name, last_name FROM user_from_song WHERE song_title = 'All Hands Against His Own';"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [66]:
query = "DROP TABLE song_length"
try:
    session.execute(query)
except Exception as e:
    print("Error dropping table {}".format(e))
    
query = "DROP TABLE user_session"
try:
    session.execute(query)
except Exception as e:
    print("Error dropping table {}".format(e))

query = "DROP TABLE user_from_song"
try:
    session.execute(query)
except Exception as e:
    print("Error dropping table {}".format(e))

### Close the session and cluster connection¶

In [67]:
session.shutdown()
cluster.shutdown()